In [10]:
from __future__ import print_function

%matplotlib inline
%load_ext autoreload
%autoreload 2


import os
import sys
import numpy as np
import matplotlib.pyplot as plt

from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model

sys.path.append('..')
from utils import display_examples, RotNetDataGenerator, angle_error, display_examples_justangle
from data.street_view import get_filenames

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load file names

In [3]:
street_view_dir = os.path.join('..', 'data', 'street_view')
# street_view_dir = 'G:/VScode/RotNet/data/test_examples'

if os.path.exists(street_view_dir):
    test_filenames = get_filenames(street_view_dir)[1]
else:
    test_examples_path = os.path.join('..', 'data', 'test_examples')
    test_filenames = [
        os.path.join(test_examples_path, f) for f in os.listdir(test_examples_path)
    ]
test_filenames

['..\\data\\test_examples\\0-1.jpg',
 '..\\data\\test_examples\\0-2.jpg',
 '..\\data\\test_examples\\0-3.jpg',
 '..\\data\\test_examples\\1-1.jpg',
 '..\\data\\test_examples\\1-2.jpg',
 '..\\data\\test_examples\\2-1.jpg',
 '..\\data\\test_examples\\2-2.JPG',
 '..\\data\\test_examples\\2-3.jpg',
 '..\\data\\test_examples\\3-1.jpg',
 '..\\data\\test_examples\\3-2.jpg']

## Load model

In [4]:
model_location = os.path.join('..', 'models', 'rotnet_street_view_resnet50_keras2.hdf5')
model = load_model(model_location, custom_objects={'angle_error': angle_error})

g:\VScode\venv-rotnet\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## Evaluate model

In [6]:
from keras.optimizers import SGD

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01, momentum=0.9),
              metrics=[angle_error])



In [9]:
model_location = os.path.join('..', 'models', 'rotnet_street_view_resnet50_keras2_compailed.hdf5')
model.save(model_location)

In [ ]:
batch_size = 64
out = model.evaluate_generator(
    RotNetDataGenerator(
        test_filenames,
        input_shape=(224, 224, 3),
        batch_size=batch_size,
        preprocess_func=preprocess_input,
        crop_center=True,
        crop_largest_rect=True,
        shuffle=True
    ),
    steps=len(test_filenames) / batch_size
)

print('Test loss:', out[0])
print('Test angle error:', out[1])

In [8]:
angle = display_examples_justangle(
    model, 
    test_filenames,
    num_images=10,
    size=(224, 224),
    crop_center=True,
    crop_largest_rect=True,
    preprocess_func=preprocess_input,
)

# 角度を基に回転予測

angle[(angle>315) | (angle<=45)] = 0
angle[(angle>45) & (angle<=135)] = 1
angle[(angle>135) & (angle<=225)] = 2
angle[(angle>225) & (angle<=315)] = 3

print(angle)


[355 351 355  97  50 173 175 184 248 325]
[0 0 0 1 1 2 2 2 3 0]


## Display examples

In [ ]:
num_images = 5

display_examples(
    model, 
    test_filenames,
    num_images=num_images,
    size=(224, 224),
    crop_center=True,
    crop_largest_rect=True,
    preprocess_func=preprocess_input,
)